In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from datetime import datetime, timedelta

import pylab

In [2]:
datapath = '../data/'

In [3]:
import glob
files = glob.glob(datapath+'*.csv')
files[0]

'../data/FC3DC207-9459-4CBE-8AB5-0E0AC77D5FEC.csv'

In [4]:
dfs = [pd.read_csv(f) for f in files]
dfs[1] = dfs[1].rename(columns={" timestamp ": "timestamp"})

In [5]:
for df in dfs:
    df['mag'] = np.sqrt(df['x']**2+df['y']**2+df['z']**2)
    df['time'] = pd.to_datetime(df['timestamp'], unit='ms')
dfs[0].head(5)

,timestamp,type,lat,lon,height,accuracy,speed,bearing,x,y,z,mag,time
0,1.431035e+12,gps,41.978753,-87.893018,194.760063,10.0,14.36,139.570312,NaN,NaN,NaN,NaN,2015-05-07 21:46:58.761430
1,1.431035e+12,accelerometer,NaN,NaN,NaN,NaN,NaN,NaN,-0.075974,-0.448395,-0.867188,0.979206,2015-05-07 21:46:58.834430
2,1.431035e+12,accelerometer,NaN,NaN,NaN,NaN,NaN,NaN,-0.079025,-0.435837,-0.919876,1.020966,2015-05-07 21:46:58.932430
3,1.431035e+12,accelerometer,NaN,NaN,NaN,NaN,NaN,NaN,-0.069183,-0.456039,-0.921722,1.030694,2015-05-07 21:46:59.030430
4,1.431035e+12,accelerometer,NaN,NaN,NaN,NaN,NaN,NaN,-0.059341,-0.456772,-0.896149,1.007593,2015-05-07 21:46:59.129430


In [6]:
def prepare_dfs(dfs,window):
    sorted_dfs = []
    rolling_mean = []
    rolling_std = []
    for df in dfs:
        _temp = df.sort_values(by='time')
        _temp = _temp.set_index(['time'])

        _mean = _temp.rolling(window).mean()
        _std = _temp.rolling(window).std()
    
        sorted_dfs.append(_temp)
        rolling_mean.append(_mean)
        rolling_std.append(_std)
    
    return sorted_dfs, rolling_mean, rolling_std




In [7]:
def Find_events(dfs, variable, window):
    events_df = pd.DataFrame()
    drivers = []
    for i in range(0,25):
        test = dfs[i].sort_values(by='time')
        test = test.set_index(['time'])

        test2 = test.rolling(window).mean()
        test2_std = test.rolling(window).std()

        for j in range(0,len(test.index)):
            if test[variable][j] > (test2[variable][j]+(2.5*test2_std[variable][j])):
                if test[variable][j] > (test[variable].mean()+(5.0*test[variable].std())):
                    print(i,test2.index[j],test[variable][j],test2[variable][j],test2_std[variable][j],test[variable].max(),test[variable].mean(),test[variable].std() )
                    events_df = events_df.append(test.iloc[j])
                    drivers.append(i)
    events_df['driver']=drivers
    return events_df
                

In [8]:
def Gather_info(event, dfs, gps, time):
    
    speed_mean_before=[]
    speed_mean_after =[]
    acc_mean_before=[]
    acc_mean_after =[]    
    distance_sum_after = []
    bearing_range_before = []
    bearing_range_after = []
    
    for i in range(0,len(event.index)):
        
        before = dfs[event.driver[i]][ (dfs[event.driver[i]].index> (event.index[i]-timedelta(seconds=time) )) & 
                       (dfs[event.driver[i]].index< (event.index[i]))]

        after = dfs[event.driver[i]][ (dfs[event.driver[i]].index> (event.index[i])) & 
                       (dfs[event.driver[i]].index< (event.index[i]+timedelta(seconds=time) ))]
        
        speed_mean_before.append(before.speed.mean())
        speed_mean_after.append(after.speed.mean())
        acc_mean_before.append(before.mag.mean())
        acc_mean_after.append(after.mag.mean())
        
        
        gps_after = gps[event.driver[i]][ gps[event.driver[i]].index> (event.index[i])]
        distance_sum_after.append(gps_after.distance.sum())
        
        gps_before = gps[event.driver[i]][ (gps[event.driver[i]].index> (event.index[i]-timedelta(seconds=10) )) & 
                       (gps[event.driver[i]].index< (event.index[i]))]
        
        gps_after = gps[event.driver[i]][ (gps[event.driver[i]].index> (event.index[i])) & 
                       (gps[event.driver[i]].index< (event.index[i]+timedelta(seconds=10) ))]
        
        bearing_range_before.append(gps_before.bearing.max() - gps_before.bearing.min())
        bearing_range_after.append(gps_after.bearing.max() - gps_after.bearing.min())
        if event.driver[i] == 3:
            print(gps_before.bearing.max(),gps_before.bearing.min())
            print(gps_after.bearing.max(),gps_after.bearing.min())
            print(bearing_range_before, bearing_range_after)
        
        
    event['speed_before'] = speed_mean_before
    event['speed_after']  = speed_mean_after     
    event['mag_before']   = acc_mean_before    
    event['mag_after']    = acc_mean_after
    event['distance_after'] = distance_sum_after
    event['bearing_before'] = bearing_range_before
    event['bearing_after'] = bearing_range_after
    
    return event
    
    
    

In [9]:
def Plot_crash(event, raw_profile, rolling_mean):
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9,9))
    
    ax1.plot(raw_profile.index,raw_profile.mag)
    ax1.plot(rolling_mean.index, rolling_mean.mag)
    ax1.plot(event.index,event.mag, marker='*',markersize=20,c='k')
 
    ax2.plot(raw_profile.index,raw_profile.mag)
    ax2.plot(rolling_mean.index, rolling_mean.mag)
    ax2.plot(event.index,event.mag, marker='*',markersize=20,c='k')
    ax2.set_xlim([event.index - timedelta(seconds=30),event.index + timedelta(seconds=30)])
             
    speed_df = raw_profile[['speed','lat','lon','height']].dropna(axis=0)
    ax3.plot(speed_df.index,speed_df.speed)
    ax3.plot([event.index,event.index],[speed_df.speed.min(),speed_df.speed.max()],c='k')
    
    ax4.plot(speed_df.index,speed_df.speed)
    ax4.plot([event.index,event.index],[speed_df.speed.min(),speed_df.speed.max()],c='k')
    ax4.set_xlim([event.index - timedelta(seconds=30),event.index + timedelta(seconds=30)])
    
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-1, event.speed_before[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-3, event.speed_after[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-5, event.mag_before[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-7, event.mag_after[0])
    
    
    
    
    plt.show()

In [10]:
def Calc_distance(dfs):
    new_df = []
    for df in dfs:
        _df = df[['lat','lon','bearing']].dropna(axis=0)
        distance = []
        distance.append(0.)
        summed_distance = []
        summed_distance.append(0.)
        for i in range(1,len(_df.index)):
            lat1 = _df.lat.iloc[i-1]
            lat2 = _df.lat.iloc[i]
            lon1 = _df.lon.iloc[i-1]
            lon2 = _df.lon.iloc[i]

            a = (np.sin((lat2 - lat1)/2))**2 + (np.cos(lat1) * np.cos(lat2) * (np.sin((lon2 - lon1)/2))**2)
            c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1 - a))
            #sofar = np.sum(distance)
            d = (6371.0*c)
            distance.append(d)
            summed_distance.append(d + summed_distance[i-1])
        _df['distance'] = distance
        _df['summed_distance'] = summed_distance
        new_df.append(_df)
    return new_df
        

In [11]:
new_dfs = Calc_distance(dfs)
sorted_dfs, rolling_mean, rolling_std = prepare_dfs(dfs,'1s')
gps_dfs = Calc_distance(sorted_dfs)
event = Find_events(dfs,'mag','1s')
event = Gather_info(event, sorted_dfs, gps_dfs, 30)

0 2015-05-07 21:51:56.474430 2.54167704146 1.17974123545 0.45567960114 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:03:24.617430 2.08860835485 1.04233454617 0.385983634763 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:06:09.797430 1.82572779139 1.01618358011 0.313007572475 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:06:15.527430 2.29722471165 1.16797535874 0.44715235887 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:07:41.591430 1.8441849796 1.11184848227 0.26025380551 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:07:50.061430 2.01246792075 1.10545868224 0.317182486423 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:07:56.386430 1.89339023558 1.1068762322 0.297349319674 3.07437785147 1.0252136609824674 0.15636141511869553
0 2015-05-07 22:09:34.652430 2.32257670361 1.11850598661 0.442537775686 3.07437785147 1.0252136609824674 0.15636141511869553
0 201

6 2015-03-19 21:32:49.043870 2.58414834802 1.0708769514 0.512740383164 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:33:12.337870 6.05384196873 1.9037404429 1.61619925288 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:33:20.094870 3.18046900325 1.06286461495 0.744007587403 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:34:46.227870 5.95807457928 1.49863309393 1.66046545005 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:34:55.142870 2.60377428558 1.0749924848 0.589832295365 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:36:18.307870 2.91825460083 1.23372030409 0.602640807496 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:36:22.383870 2.53515295875 1.13820692773 0.480446886636 7.49083739885 1.0310558508151255 0.29910192886533754
6 2015-03-19 21:37:11.864870 3.77458337564 1.15076553574 0.950324343404 7.49083739885 1.0310558508151255 0.29910192886533754
6 201

10 2015-06-20 20:48:37.808120 1.53395653777 1.06481701054 0.173672662096 7.53591375767 1.0012670093424914 0.09666070356679585
10 2015-06-20 20:51:31.474120 1.77520996022 1.05162125838 0.245803637386 7.53591375767 1.0012670093424914 0.09666070356679585
10 2015-06-20 20:55:57.826120 2.86325248522 1.10732410511 0.595901408721 7.53591375767 1.0012670093424914 0.09666070356679585
10 2015-06-20 20:56:33.981120 2.86523081957 1.15090737958 0.628058963938 7.53591375767 1.0012670093424914 0.09666070356679585
11 2015-06-24 16:43:24.523000 2.89927258751 1.17815301284 0.619605273033 8.31088170278 1.0108245282831234 0.16329734169136015
11 2015-06-24 16:43:24.622000 4.25198149962 1.47155833753 1.10981857872 8.31088170278 1.0108245282831234 0.16329734169136015
11 2015-06-24 16:47:33.709000 4.3835231685 1.29313549669 1.03662191061 8.31088170278 1.0108245282831234 0.16329734169136015
11 2015-06-24 16:48:06.734000 2.06536171627 1.10105799958 0.320811800599 8.31088170278 1.0108245282831234 0.1632973416913

18 2015-01-12 10:38:24.414680 1.84352236688 1.11777631535 0.282562403446 8.34663307494 1.0048569324620829 0.1614981283026544
18 2015-01-12 10:38:24.915680 8.34663307494 1.88179399115 2.29791532763 8.34663307494 1.0048569324620829 0.1614981283026544
18 2015-01-12 10:38:43.242680 2.09030721772 1.0966287014 0.353355872822 8.34663307494 1.0048569324620829 0.1614981283026544
18 2015-01-12 10:41:55.740680 2.17833364361 1.21443494973 0.379008615089 8.34663307494 1.0048569324620829 0.1614981283026544
19 2015-04-19 22:24:30.271430 2.49711312865 1.2612914292 0.443582107095 7.50492170969 1.0072596975762942 0.11697337362592983
19 2015-04-19 22:24:30.473430 7.50492170969 1.87909766833 2.03470050241 7.50492170969 1.0072596975762942 0.11697337362592983
20 2015-06-18 12:32:29.150130 1.97599232768 1.06678556214 0.309572346095 2.64093953634 1.0039679141434383 0.07250826861818559
20 2015-06-18 12:32:36.346130 1.38613509824 1.04802267526 0.11913320842 2.64093953634 1.0039679141434383 0.07250826861818559
2

23 2015-05-12 18:32:51.164110 3.40902718337 1.32139002159 0.738418223422 4.77366504831 1.033425330132297 0.20571067267913742
23 2015-05-12 18:33:19.525110 2.13298461673 1.07295878086 0.383253515792 4.77366504831 1.033425330132297 0.20571067267913742
24 2015-05-17 16:00:56.685600 1.43392990155 1.05871759464 0.145996721065 2.34334195409 1.0037631082889007 0.0754920804611185
24 2015-05-17 16:02:04.529600 1.54646006132 1.06284004269 0.164660183373 2.34334195409 1.0037631082889007 0.0754920804611185
24 2015-05-17 16:02:05.930600 1.44640083262 0.97664490729 0.172124365039 2.34334195409 1.0037631082889007 0.0754920804611185
24 2015-05-17 16:04:23.392600 2.34334195409 1.14391664585 0.398124444097 2.34334195409 1.0037631082889007 0.0754920804611185
24 2015-05-17 16:07:40.892600 1.75867015944 1.06945816508 0.228593682628 2.34334195409 1.0037631082889007 0.0754920804611185
24 2015-05-17 16:07:50.568600 1.38418045437 1.02564702237 0.131286828103 2.34334195409 1.0037631082889007 0.0754920804611185


In [12]:
def Plot_crash(event, raw_profile, rolling_mean, gps_dfs):
    f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(9,13.5))
    
    ax1.plot(raw_profile.index,raw_profile.mag)
    ax1.plot(rolling_mean.index, rolling_mean.mag)
    ax1.plot(event.index,event.mag, marker='*',markersize=20,c='k')
    ax1.set_title("Acceleration - Full profile")
 
    ax2.plot(raw_profile.index,raw_profile.mag)
    ax2.plot(rolling_mean.index, rolling_mean.mag)
    ax2.plot(event.index,event.mag, marker='*',markersize=20,c='k')
    ax2.set_xlim([event.index - timedelta(seconds=30),event.index + timedelta(seconds=30)])
    ax2.set_title("Acceleration - -30sec to +30sec of event")
             
    speed_df = raw_profile[['speed','lat','lon','height']].dropna(axis=0)
    ax3.plot(speed_df.index,speed_df.speed)
    ax3.plot([event.index,event.index],[speed_df.speed.min(),speed_df.speed.max()],c='k')
    ax3.set_title("Speed - Full profile")
    
    ax4.plot(speed_df.index,speed_df.speed)
    ax4.plot([event.index,event.index],[speed_df.speed.min(),speed_df.speed.max()],c='k')
    ax4.set_xlim([event.index - timedelta(seconds=30),event.index + timedelta(seconds=30)])
    ax4.set_title("Speed - -30sec to +30sec of event")
    
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-1, event.speed_before[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-3, event.speed_after[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-5, event.mag_before[0])
    ax4.text(event.index+timedelta(seconds=5), speed_df.speed.max()-7, event.mag_after[0])
    
    ax5.plot(gps_dfs.index, gps_dfs.distance)
    ax5.plot([event.index,event.index],[gps_dfs.distance.min(),gps_dfs.distance.max()],c='k')
    ax5.set_title("Distance - Full profile")
    ax5.set_xlim([event.index - timedelta(seconds=30),gps_dfs.index.max()])
    ax5.text(event.index+timedelta(seconds=5), gps_dfs.distance.max(), event.distance_after[0])
    
    #ax6.plot(gps_dfs.index, gps_dfs.distance)
    #ax6.plot([event.index,event.index],[gps_dfs.distance.min(),gps_dfs.distance.max()],c='k')
    #ax6.set_xlim([event.index - timedelta(seconds=30),gps_dfs.index.max()])
    
    ax6.plot(gps_dfs.index, gps_dfs.bearing)
    ax6.plot([event.index,event.index],[gps_dfs.bearing.min(),gps_dfs.bearing.max()],c='k')
    ax6.set_xlim([event.index - timedelta(seconds=30),event.index + timedelta(seconds=30)])
    ax6.set_title("Bearing - -30sec to +30sec of event")
    ax6.text(event.index+timedelta(seconds=5), 250, event.bearing_before[0])
    ax6.text(event.index+timedelta(seconds=5), 230, event.bearing_after[0])
    
    pylab.savefig('../figs/'+str(event.iloc[0:1].driver[0])+'_'+str(i)+'.png', bbox_inches=0)
    plt.close()
    
    
    
    
    #plt.show()

In [13]:
#for i in range(0,len(event.index)):
#    Plot_crash(event.iloc[i:i+1],sorted_dfs[event.driver[i]], rolling_mean[event.driver[i]], gps_dfs[event.driver[i]])



In [14]:
event.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 290 entries, 2015-05-07 21:51:56.474430 to 2015-05-17 16:10:46.173600
Data columns (total 20 columns):
accuracy          0 non-null float64
bearing           0 non-null float64
height            0 non-null float64
lat               0 non-null float64
lon               0 non-null float64
mag               290 non-null float64
speed             0 non-null float64
timestamp         290 non-null float64
type              290 non-null object
x                 290 non-null float64
y                 290 non-null float64
z                 290 non-null float64
driver            290 non-null int64
speed_before      273 non-null float64
speed_after       269 non-null float64
mag_before        290 non-null float64
mag_after         290 non-null float64
distance_after    290 non-null float64
bearing_before    265 non-null float64
bearing_after     264 non-null float64
dtypes: float64(18), int64(1), object(1)
memory usage: 57.6+ KB


In [15]:
event[event.speed_before>event.speed_after].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 134 entries, 2015-05-07 21:51:56.474430 to 2015-05-17 16:04:23.392600
Data columns (total 20 columns):
accuracy          0 non-null float64
bearing           0 non-null float64
height            0 non-null float64
lat               0 non-null float64
lon               0 non-null float64
mag               134 non-null float64
speed             0 non-null float64
timestamp         134 non-null float64
type              134 non-null object
x                 134 non-null float64
y                 134 non-null float64
z                 134 non-null float64
driver            134 non-null int64
speed_before      134 non-null float64
speed_after       134 non-null float64
mag_before        134 non-null float64
mag_after         134 non-null float64
distance_after    134 non-null float64
bearing_before    134 non-null float64
bearing_after     134 non-null float64
dtypes: float64(18), int64(1), object(1)
memory usage: 22.0+ KB


In [16]:
event[(event.speed_before>event.speed_after) & (event.distance_after<10) & ((event.bearing_after - event.bearing_before)>100)].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16 entries, 2015-06-20 22:20:58.805070 to 2015-05-12 18:29:56.465110
Data columns (total 20 columns):
accuracy          0 non-null float64
bearing           0 non-null float64
height            0 non-null float64
lat               0 non-null float64
lon               0 non-null float64
mag               16 non-null float64
speed             0 non-null float64
timestamp         16 non-null float64
type              16 non-null object
x                 16 non-null float64
y                 16 non-null float64
z                 16 non-null float64
driver            16 non-null int64
speed_before      16 non-null float64
speed_after       16 non-null float64
mag_before        16 non-null float64
mag_after         16 non-null float64
distance_after    16 non-null float64
bearing_before    16 non-null float64
bearing_after     16 non-null float64
dtypes: float64(18), int64(1), object(1)
memory usage: 2.6+ KB


In [17]:
event_cut = event[(event.speed_before>event.speed_after) & (event.distance_after<10) & ((event.bearing_after - event.bearing_before)>100)]

In [18]:
for i in range(0,len(event_cut.index)):
    Plot_crash(event_cut.iloc[i:i+1],sorted_dfs[event_cut.driver[i]], rolling_mean[event_cut.driver[i]], gps_dfs[event_cut.driver[i]])


